In [1]:
import torch
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub


c:\Users\Admin\anaconda3\envs\RAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Loader = PyPDFLoader
FILE_PATH = "./AIO2024_Module01_Project_YOLOv10_Description.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()
documents

[Document(metadata={'source': './AIO2024_Module01_Project_YOLOv10_Description.pdf', 'page': 0}, page_content='AI VIET NAM – AI COURSE 2024\nKiểm tra tuân thủ đội mũ bảo vệ với YOLOv10\nDinh-Thang Duong, Quang-Vinh Dinh\nNgày 22 tháng 6 năm 2024\nI. Giới thiệu\nObject Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh\nvực Computer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng\ntrong một tấm ảnh. Đây là một trong những bài toán quan trọng và phức tạp trong Computer\nVision, với ứng dụng rộng rãi từ nhận diện khuôn mặt, nhận dạng biển số xe, đến theo dõi đối\ntượng trong video và tự động lái xe.\nHình 1: Chương trình phát hiện đối tượng có mang mũ bảo hiểm.\nTrong project này, chúng ta sẽ xây dựng một chương trình phát hiện các nhân viên có đeo mũ\nbảo vệ trong công trường hay không? Mô hình mà chúng ta sử dụng sẽ là mô hình YOLOv10.\nTheo đó, Input và Output của chương trình là:\n•Input:Một tấm ảnh.\n•Output: Tọa độ (bounding bo

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
text_splitter

In [4]:
docs = text_splitter.split_documents(documents)

print("Number of sub-documents:", len(docs))
print(docs[0])


Number of sub-documents: 29
page_content='AI VIET NAM – AI COURSE 2024
Kiểm tra tuân thủ đội mũ bảo vệ với YOLOv10
Dinh-Thang Duong, Quang-Vinh Dinh
Ngày 22 tháng 6 năm 2024
I. Giới thiệu
Object Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh
vực Computer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng
trong một tấm ảnh. Đây là một trong những bài toán quan trọng và phức tạp trong Computer
Vision, với ứng dụng rộng rãi từ nhận diện khuôn mặt, nhận dạng biển số xe, đến theo dõi đối
tượng trong video và tự động lái xe.
Hình 1: Chương trình phát hiện đối tượng có mang mũ bảo hiểm.
Trong project này, chúng ta sẽ xây dựng một chương trình phát hiện các nhân viên có đeo mũ
bảo vệ trong công trường hay không? Mô hình mà chúng ta sử dụng sẽ là mô hình YOLOv10.
Theo đó, Input và Output của chương trình là:
•Input:Một tấm ảnh.
•Output: Tọa độ (bounding box) của các nhân viên và phần mũ bảo hiểm.' metadata={'source': './AIO2024_Module01_

In [5]:
embedding = HuggingFaceEmbeddings()

In [6]:
vector_db = Chroma.from_documents(documents=docs, embedding=embedding)

retriever = vector_db.as_retriever()


In [7]:
result = retriever.invoke("What is YOLO?")

print("Number of relevant documents:", len(result))
result

Number of relevant documents: 4


[Document(metadata={'page': 12, 'source': './AIO2024_Module01_Project_YOLOv10_Description.pdf'}, page_content='AI VIETNAM (AIO2024) aivietnam.edu.vn\nHình 19: Thư mục chứa bộ dữ liệu\nQuan sát thư mục, có thể thấy bộ dữ liệu này đã được gán nhãn và đưa vào format cấu\ntrúc dữ liệu training theo yêu cầu của YOLO. Vì vậy, chúng ta sẽ không cần thực hiện\nbước chuẩn bị dữ liệu ở bài này.\n2.Cài đặt và import các thư viện cần thiết: Tương tự như phần trước, các bạn chạy\ncác đoạn code sau để cài đặt các gói thư viện để sử dụng được YOLOv10:\n1!git clone https :// github .com/THU -MIG/ yolov10 .git\n2%cd yolov10\n3!pip install -q -r requirements .txt\n4!pip install -e .\n3.Khởi tạo mô hình YOLOv10: Chúng ta sẽ khởi tạo mộ hình YOLOv10 với phiên\nbảnnano (n) từ trọng số đã được huấn luyện trên bộ dữ liệu COCO. Để tải trọng số\nyolov10n.pt, các bạn chạy đoạn code sau:\n1! wget https :// github .com/THU -MIG/ yolov10 / releases / download /v1 .1/\nyolov10n .pt\nSau đó, để khởi tạo mô hình từ t

In [8]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [9]:
MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.83s/it]
c:\Users\Admin\anaconda3\envs\RAG\lib\site-packages\transformers\generation\configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
c:\Users\Admin\anaconda3\envs\RAG\lib\site-packages\transformers\generation\configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fix

In [10]:
model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline
)


In [11]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

USER_QUESTION = "YOLOv10 là gì?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer:')[1].strip()
print(answer)


c:\Users\Admin\anaconda3\envs\RAG\lib\site-packages\transformers\generation\configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Admin\anaconda3\envs\RAG\lib\site-packages\transformers\generation\configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\Admin\anaconda3\envs\RAG\lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention

YOLOv10 là một phiên bản nano của mô hình dự đoán ảnh YOLOv10, được tạo bằng cách tải trọng số của mô hình được huấn luyện trên dữ liệu COCO và khởi tạo tại Google Colab bằng cách cài đặt các thư viện cần thiết và tải trọng số của pre-trained models.
